In [197]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.datasets import make_regression

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [231]:
from src.models import Question, Answer, AnswerRelation
from src.main_util import load_questions, extract_features, build_df, split_train_test, build_question_answer_df, build_question_answer_dict

In [188]:
print('Loading questions..')
questions = load_questions('./MCQ/')

Loading questions..


In [189]:
print('Extracting features..')
extract_features(questions)

  0%|          | 5/5852 [00:00<01:57, 49.65it/s]

Extracting features..


100%|██████████| 5852/5852 [02:11<00:00, 44.63it/s]


In [251]:
train, test = split_train_test(questions, train_ratio=0.75, make_correct_answer_first=True, swap_multiply_test=False, question_answer_pairs=True)

In [254]:
df_train = build_question_answer_df(train, include_metadata=False)

In [335]:
label_col = 'label'
X, y = df_train.drop(label_col, axis=1), df_train[label_col]
clf = RandomForestClassifier(max_depth=3, random_state=0, class_weight='balanced')
clf.fit(X, y)
print(df_train.drop(label_col, axis=1).columns)
print(clf.feature_importances_)

Index([u'a_all_or_none', u'a_average_frequency', u'a_certainty_count',
       u'a_char_count', u'a_common_synonyms_with_question_count',
       u'a_common_words_with_question_count', u'a_similarity_with_question',
       u'a_stopword_count', u'a_title_count', u'a_without_stopword_count',
       u'a_word_count', u'q_average_frequency', u'q_char_count',
       u'q_is_completion', u'q_is_question', u'q_num_answers',
       u'q_stopword_count', u'q_title_count', u'q_without_stopword_count',
       u'q_word_count'],
      dtype='object')
[ 0.00497264  0.1167106   0.23495039  0.2055002   0.00960168  0.01456107
  0.00872891  0.05997293  0.12826165  0.06339339  0.0968523   0.01602133
  0.019259    0.          0.          0.          0.          0.01006512
  0.0111488   0.        ]


In [358]:
true_positives = 0
false_positives = 0
true_negatives = 0
false_negatives = 0
positives = 0
negatives = 0
correct = [True, False, False, False]

for qs in test:
    new_qs = [build_question_answer_dict(q_a, q_a.answers[0], include_metadata=False) for q_a in qs]
    new_qs = sorted(new_qs, key=lambda q: q[label_col], reverse=True)
    df = pd.DataFrame(new_qs).drop(label_col, axis=1)
    res = clf.predict(df)
    
    if res[0] == True:
        true_positives += 1
    else:
        false_negatives += 1
    positives += 1
        
    for r in res[1:]:
        if r == True:
            false_positives += 1
        else:
            true_negatives += 1
    negatives += 3
    
total = positives + negatives

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


In [337]:
float(false_positives) / total

0.2472658920027341

In [338]:
float(false_negatives) / total

0.15379357484620643

In [339]:
float(true_positives) / total

0.09620642515379357

In [340]:
float(true_negatives) / total

0.5027341079972659